In [1]:
from __code import system
system.System.select_working_dir()
from __code.__all import custom_style
custom_style.style()

** Using Debugging Mode! **


HBox(children=(Label(value='Select Working Folder', layout=Layout(width='20%')), Select(options=('IPTS_18813',), value='IPTS_18813')))

# Create list of file name vs time stamp 

To do so, run the following notebook -> [create_list_of_file_name_vs_time_stamp](create_list_of_file_name_vs_time_stamp.ipynb)

# Sort files according to time stamp 

In [2]:
import ipywe.fileselector
import pandas as pd

In [3]:
file_name_ui = ipywe.fileselector.FileSelectorPanel(instruction='Select file created by the create_list_of_file_name_vs_time_stamp',
                                                   start_dir = system.System.get_working_dir())
file_name_ui.show()

HTML(value='Please wait...')

VBox(children=(Label(value='Select file created by the create_list_of_file_name_vs_time_stamp', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Users/j35/IPTS/IPTS_18813', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(Select(description='Select', layout=Layout(width='750px'), options=(' .', ' ..', ' .DS_Store                        |     Tue Jan 30 12:58:36 2018', ' Sample_2                         |     Directory', ' Sample_2_timestamp_infos.txt     |     Tue Jan 30 09:32:03 2018'), value=' .'), HBox(children=(Button(description='Enter directory', layout=Layout(margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(margin='5px 40px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px'))))

In [4]:
list_of_filename_vs_timestamp = file_name_ui.selected
pd_file_vs_timestamp = pd.read_csv(list_of_filename_vs_timestamp)
pd_file_vs_timestamp.columns = ['filename','time_stamp_s','time_human_readable','time_offset_s']
pd_file_vs_timestamp.head()

,filename,time_stamp_s,time_human_readable,time_offset_s
0,/Users/j35/IPTS/IPTS_18813/Sample_2/20180116_S...,1.516139e+09,2018-01-16 16:48:35,0.000000
1,/Users/j35/IPTS/IPTS_18813/Sample_2/20180116_S...,1.516139e+09,2018-01-16 16:49:10,34.970798
2,/Users/j35/IPTS/IPTS_18813/Sample_2/20180116_S...,1.516139e+09,2018-01-16 16:49:45,69.884376
3,/Users/j35/IPTS/IPTS_18813/Sample_2/20180116_S...,1.516139e+09,2018-01-16 16:50:20,104.803482
4,/Users/j35/IPTS/IPTS_18813/Sample_2/20180116_S...,1.516139e+09,2018-01-16 16:50:55,139.721396


In [5]:
list_of_files = pd_file_vs_timestamp['filename']

### Figure out how we are going to parse the file names

re to use '^\s_[\s^_]_[\s^_]_\d{4}_\d{4}.tiff$'



In [7]:
import os, re

In [8]:
my_re = r"\w*_(?P<temperature>\w+)_(?P<pressure>\w+)_\d{4}_(?P<digit>\d{4}).tiff$"
file_name = os.path.basename(list_of_files[0])
print(file_name)

20180116_Sample2_Methane_293K_89Bar_0030_0024.tiff


In [9]:
m = re.match(my_re, file_name)

In [10]:
m.group('temperature')

'293K'

In [11]:
from collections import OrderedDict

In [18]:
my_re = r"\w*_(?P<temperature>\w+)_(?P<pressure>\w+)_\d{4}_(?P<digit>\d{4}).tiff$"
exp_dict = OrderedDict()
_new_exp = {'list_of_files': [], 'T':'', 'P':''}

previous_T = ''
previous_P = ''

exp_index = 0

for _index, _file in enumerate(list_of_files):
    _file = os.path.basename(_file)
    m = re.match(my_re, _file)
    if m is None:
        continue

    if m:
        _T = m.group('temperature')
        _P = m.group('pressure')
    else:
        _T = 'N/A'
        _P = 'N/A'

    # first entry
    if _new_exp['list_of_files'] == []:
        _new_exp['list_of_files'] = [os.path.basename(_file)]
        
        _new_exp['T'] = _T
        previous_T = _T
            
        _new_exp['P'] = _P
        previous_P = _P
            
    # after first entry
    else:
        
        if (_T == previous_T) and (_P == previous_P):
            # found the same T and P, just append file to list of files
            _new_exp['list_of_files'].append(_file)
            
        else:
            # we found a new T or new P, save previous dict and initialization of new one
            exp_dict[str(exp_index)] = _new_exp
            exp_index += 1
            _new_exp = {'list_of_files': [], 'T':'', 'P':''}

            # start recording new entry
            _new_exp['list_of_files'] = [os.path.basename(_file)]
        
            _new_exp['T'] = _T
            previous_T = _T
            
            _new_exp['P'] = _P
            previous_P = _P
        
    # last entry, force the save of the dictionary
    if _index == len(list_of_files)-1:
        exp_dict[str(exp_index)] = _new_exp
        
        
        
            
            
    
    
    

In [19]:
exp_dict

OrderedDict([('0',
              {'P': '89Bar',
               'T': '293K',
               'list_of_files': ['20180116_Sample2_Methane_293K_89Bar_0030_0024.tiff',
                '20180116_Sample2_Methane_293K_89Bar_0030_0025.tiff',
                '20180116_Sample2_Methane_293K_89Bar_0030_0026.tiff',
                '20180116_Sample2_Methane_293K_89Bar_0030_0027.tiff',
                '20180116_Sample2_Methane_293K_89Bar_0030_0028.tiff']}),
             ('1',
              {'P': '60Bar',
               'T': '293K',
               'list_of_files': ['20180116_Sample2_Methane_293K_60Bar_0030_0029.tiff',
                '20180116_Sample2_Methane_293K_60Bar_0030_0030.tiff',
                '20180116_Sample2_Methane_293K_60Bar_0030_0031.tiff',
                '20180116_Sample2_Methane_293K_60Bar_0030_0032.tiff',
                '20180116_Sample2_Methane_293K_60Bar_0030_0033.tiff']}),
             ('2',
              {'P': '30Bar',
               'T': '293K',
               'list_of_files': 

SAndbox

In [74]:
my_re_sandbox = r"\w*_(?P<temperature>\w+)_(?P<pressure>\w+)_\d{4}_(?P<digit>\d{4}).tiff$"
file_name = os.path.basename(list_of_files[0])
file_name = "20180116_Sample2_Methane_293K_89Bar_0030_0024.tiff"
print(file_name)

20180116_Sample2_Methane_293K_89Bar_0030_0024.tiff


In [75]:
m = re.match(my_re_sandbox, file_name)
print(m)

<_sre.SRE_Match object; span=(0, 50), match='20180116_Sample2_Methane_293K_89Bar_0030_0024.tif>


In [77]:
print("digit is: {}".format(m.group('digit')))
print("pressure is: {}".format(m.group('pressure')))
print("temperature is: {}".format(m.group('temperature')))

digit is: 0024
pressure is: 89Bar
temperature is: 293K
